In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from scipy import stats
from code.python.mpmath_integration import quad_phi

from mpmath import mp

from code.nig import nig_cdf

In [58]:
alpha = 48.8
beta = -1.96
mu = 0
delta = 0.021
x = mu
x, mu

(0, 0)

In [53]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=300)

In [54]:
scipy_result = stats.norminvgauss.cdf(x=x, a=alpha*delta, b=beta*delta, loc=mu, scale=delta)

In [55]:
gnp_result = nig_cdf(x, alpha, beta, mu, delta)

In [56]:
mp_result

mpf('1.255348718701679174931758777929173559329880173135408724057490697243203581536465860033344056601887645232e-170')

In [57]:
gnp_result

2.2122746392077013e-175

In [39]:
mp.nprint(abs(mp_result / scipy_result - 1), 4)

ZeroDivisionError: 

In [40]:
mp.nprint(abs(mp_result / gnp_result - 1), 4)

1.0


### Test

In [9]:
import itertools

In [10]:
xxs = np.linspace(-10, 10, 20)

alphas = np.linspace(1e-3, 100, 100)
rab = np.linspace(1e-3, 1-1e-3, 50)
betas = np.concatenate((alphas[:50] * rab, -alphas[50:] * rab))
mus = np.linspace(-10, 10, 20)
deltas = np.linspace(1e-3, 50, 20)

In [11]:
A = np.c_[alphas, betas]

In [12]:
B = np.array(list(itertools.product(mus, xxs, deltas)))

#### 1. Test x = $\mu$

In [59]:
rs = np.random.default_rng(seed=1)

alphas = rs.uniform(0.001, 100, 1000)
r = rs.uniform(0.001, 0.999, 1000)
betas = alphas * r
deltas = rs.uniform(0.001, 100, 1000)

rs = np.random.default_rng(seed=1)

alphas = rs.uniform(0.0001, 10, 500)
r = rs.uniform(0.001, 0.999, 500)
betas = alphas * r
deltas = rs.uniform(0.0001, 10, 500)

In [60]:
with open('instances_x_eq_mu_small.csv', 'a') as f:
    header = 'alpha,beta,delta,mpmath,scipy,gnp\n'
    f.write(header)

iter = 0
for alpha, beta, delta in zip(alphas, betas, deltas):
    print(f'{iter} | alpha={alpha} beta={beta} delta={delta}')
    iter += 1
    x = mu = 0
    
    # Mpmath reference
    mp_result = quad_phi(0, alpha, beta, mu, delta, digits=100)
    
    # Scipy
    scipy_result = stats.norminvgauss.cdf(x=x, a=alpha*delta, b=beta*delta, loc=mu, scale=delta)

    # GNP
    gnp_result = nig_cdf(x, alpha, beta, mu, delta)

    instance = f'{alpha},{beta},{delta},{float(mp_result):.16E},{scipy_result:.16E},{gnp_result:.16E}\n'
    
    with open('instances_x_eq_mu_small.csv', 'a') as f:
        f.write(instance)

0 | alpha=5.118265064840097 beta=2.1586812665330752 delta=5.423310782191326
1 | alpha=9.504641916889721 beta=0.25202475016780057 delta=9.678837796132974
2 | alpha=1.4416817112350655 beta=0.24313314458871524 delta=7.047853886931386
3 | alpha=9.486499606427724 beta=7.105026813023674 delta=5.452297557772693
4 | alpha=3.1183833369596536 beta=0.2645535332610986 delta=4.636803811448154
5 | alpha=4.233322157080859 beta=1.3251340813089978 delta=4.664043429436205
6 | alpha=8.277043167945035 beta=2.1164566030426837 delta=6.819999071068578
7 | alpha=4.092050443777976 beta=3.0510683916884918 delta=2.3451302372226257
8 | alpha=5.495981917361828 beta=1.9749355376646218 delta=7.154039704923569
9 | alpha=0.27568837651935935 beta=0.024264399417067494 delta=4.909948271301038
10 | alpha=7.535155735437199 beta=2.7932914060262646 delta=6.746654719160031
11 | alpha=5.38147931786146 beta=1.7606920448946168 delta=9.408623417947362
12 | alpha=3.297384191819272 beta=2.3628637161438486 delta=2.3346134725521286
1

In [13]:
iter = 0

with open('instances_x_eq_mu.csv', 'a') as f:
    header = 'alpha,beta,delta,mpmath,scipy,gnp\n'
    f.write(header)

for i in range(A.shape[0]):
    alpha, beta = A[i, :]
    for j in range(B.shape[0]):
        iter += 1
        mu, x, delta = B[j, :]

        print(f'{iter} | x={x} alpha={alpha} beta={beta} mu={mu} delta={delta}')
        # Mpmath reference
        
        mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
        
        # Scipy
        scipy_result = stats.norminvgauss.cdf(x=x, a=alpha*delta, b=beta*delta, loc=mu, scale=delta)

        # GNP
        gnp_result = nig_cdf(x, alpha, beta, mu, delta)

        instance = f'{x},{alpha},{beta},{mu},{delta},{float(mp_result):.16E},{scipy_result:.16E},{gnp_result:.16E}\n'
        
        with open('instances.csv', 'a') as f:
            f.write(instance)

1 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=0.001
2 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=2.6325263157894736
3 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=5.264052631578948
4 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=7.8955789473684215
5 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=10.527105263157894
6 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=13.158631578947368
7 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=15.790157894736842
8 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=18.42168421052632
9 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=21.05321052631579
10 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=23.684736842105263
11 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=26.31626315789474
12 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=28.947789473684214
13 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=31.579315789473686
14 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 delta=34.210842105263154
15 | x=-10.0 alpha=0.001 beta=1e-06 mu=-10.0 d

/home/gui/anaconda3/envs/python39/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1950: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  return integrate.quad(self._pdf, _a, x, args=args)[0]


62 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=2.6325263157894736
63 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=5.264052631578948
64 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=7.8955789473684215
65 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=10.527105263157894
66 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=13.158631578947368
67 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=15.790157894736842
68 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=18.42168421052632
69 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=21.05321052631579
70 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=23.684736842105263
71 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=26.31626315789474
72 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=28.947789473684214
73 | x=-6.842105263157895 alpha=0.001 beta=1e-06 mu=-10.0 delta=31.579315789473686
74 | x=-

/home/gui/anaconda3/envs/python39/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1950: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return integrate.quad(self._pdf, _a, x, args=args)[0]


142 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=2.6325263157894736
143 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=5.264052631578948
144 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=7.8955789473684215
145 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=10.527105263157894
146 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=13.158631578947368
147 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=15.790157894736842
148 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=18.42168421052632
149 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=21.05321052631579
150 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=23.684736842105263
151 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=26.31626315789474
152 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=28.947789473684214
153 | x=-2.6315789473684212 alpha=0.001 beta=1e-06 mu=-10.0 delta=31.

KeyboardInterrupt: 